# P03 NO2 and NSL

#### Michiel van der Molen, Antonija Rimac (Version 2.0, August, 2020)

Content
1. Introduction
2. Emissions
3. From Emissions to Concentrations

    3.1 Budget Equation   
    3.2 Ventilation rate    
    3.3 Tree factor    
    3.4 Road Type    
    3.5 Other complicating factors
        3.5.1 Crossings
        3.5.2 Parking movements
        3.5.3 Tunnel factor
    3.6 Probability of Exceedence of the air quality standards    
        
        
4. NSL model in practise<br>

    4.1. Background concentration<br>
    4.2. Emissions along the street segment<br>
    4.3. Contribution from nearby motorways<br>
    4.4. Total concentration<br>
    
    
5. Exercises NSL
6. Exercises Observations
7. Exercises Comparison 


## 1. Introduction

[NSL-monitoring](www.nsl-monitoring.nl) is the standardised calculation tool for the National Cooperation for Air Quality, which is an implementation of [the national law for air quality](http://wetten.overheid.nl/) (for details search for ‘Regeling Beoordeling Luchtkwaliteit 2007’). It is the official tool which local governments use to simulate the air quality in particular streets, to pinpoint problem areas, where the EU standards for air quality are exceeded, and to report to the national governments. The tool results from theoretical constraints, calibration with measurements on many different locations and the need for an objective and user-friendly reporting protocol. 

<br>
<div align=center>
<figure>
  <img src="Figure1_boxmodel.png", width="500px" height="400px">
  <figcaption> <i>Figure 1: The experimental setup</i></figcaption>
</figure>
</div>
<br>

The NSL-monitoring considers each street segment as a single box (see Figure 1). The concentration in a box depends on the difference between inflow and outflow ($F_{in}$ and $F_{out}$), emission and deposition ($E$ and $D$) and chemical production and loss ($P$ and $L$, respectively). The concentrations of the air moving in or out of the box are not coupled to the concentrations in neighbouring boxes, so there is no exchange between adjacent street segments. This is clearly a simplification, which may be violated in reality. The implicit assumption of using a box model is that the concentrations within the box are well mixed. This is also a simplication, as the concentration of pollutantant may be considerably higher on the downwind side of the street than on the upwind side. However, the NSL-monitoring is developed as a generic tool that does not use detailed meteorological information. The tool also does not pretend to simulate diurnally varying concentrations. It only simulates annual average concentrations. 

The mass balance of a substance in that box can be described using the continuity equation:

\begin{equation}
\frac{dm}{dt} = F_{in} - F_{out} + E - D + P - L
\end{equation}

The NSL-monitoring assumes that there is no loss by deposition (thus $D$ = 0), because the deposition velocity $v_d$ is very low for NO$_2$ and zero for NO. Inside the box vehicles add pollutants by emission $E$, according to the traffic intensity and the emission factors (emission per distance driven), and a few additional factors. The model assumes that there is equilibrium according to the photostationary state (NO + O$_3$ ↔ NO$_2$ + O$_2$) and no chemical production and loss of NO and NO$_2$ ($P$ = $L$ = 0). The model assumes a background concentration of pollutants in the inflowing air according to GCN maps (Grootschalige Concentratiekaart Nederland, Large-scale Concentration map of the Netherlands, provided by RIVM). The outflow of the pollutants depends on the concentration in the box and the wind speed. In the following, we will separately describe the terms $E$, $F_{in}$ and $F_{out}$.

This python notebook calculates for a specific receptor point, the concentrations of the air pollutants NO$_2$, PM$_{10}$ and PM$_{2.5}$. Calculations are based on Standard Calculation Method 1 (SCM1, NL: StandaardRekenMethode 1 (SRM1)), which is in the Dutch rule of law the prescribed method for calculating annual averages of atmospheric pollutants for receptor points that are located in urban areas.   

## 2. Emissions

The emission $E$ ($\mu$g m$^{-1}$ s$^{-1}$) in a street segment is computed from the vehicle intensity $N$ (veh day$^{-1}$) and the emission per vehicle per driven meter, called the emission factor $E^*$ (g km$^{-1}$ veh$^{-1}$) as:

\begin{equation}
E = \frac{1000}{24 \cdot 3600} N\cdot E^*.
\end{equation}

The factor $\frac{1000}{24 \cdot 3600}$ converts $N\cdot E$ from (g km$^{-1}$ day$^{-1}$) to ($\mu$g m$^{-1}$ s$^{-1}$). The traffic intensity is distributed over a fraction of light vehicles ($f_l$), a fraction of middle heavy vehicles ($f_m$) and fraction of heavy vehicles ($f_h$), each with their own emission factors ($E_l$, $E_m$, $E_h$), because trucks emit more pollutants per driven distance than passenger cars. Busses may be assigned a fraction of their own, or they may be counted as heavy vehicles. 

\begin{equation}
E^* = f_l \cdot N_l + f_m \cdot N_m + f_h \cdot N_h.
\end{equation}

The emission factors depend not only on the vehicle category, but also on their speed. The NSL-monitoring distinguishes 5 speed classes as shown in Table 1. 

<html>
<head>
<style>
table, th, td {
  border: 1px solid black; border-collapse: collapse;
}
th, td {
    padding: 15px; 
}
</style>
</head>
<body>
    
<p>Table 1: Speed types in the NSL-monitoring. The code (a) is no longer used and it is replaced by codes 92-94.</p>
<table style="width:100%">
       <colgroup>
       <col span="1" style="width: 20%;">
       <col span="1" style="width: 30%;">
       <col span="1" style="width: 50%;">
    </colgroup>    
  <tr>
    <th style="text-align:center">Speed code</th>
    <th style="text-align:center">Speed type</th> 
    <th style="text-align:center">Description</th>
  </tr>
  <tr>
    <td style="text-align:center">(a)</td>
    <td style="text-align:center">Highway</td>
    <td style="text-align:center">Typical highway traffic, v>65 km h$^{-1}$, 0.2 stops per km</td>
  </tr>
  <tr>
    <td style="text-align:center">b</td>
    <td style="text-align:center">Provincial road</td>
    <td style="text-align:center">Typical inter-communal traffic, v~60 km h$^{-1}$, 0.2 stops per km</td>
  </tr>
    <tr>
    <td style="text-align:center">e</td>
    <td style="text-align:center">Urban traffic with reduced congestion</td>
    <td style="text-align:center">Urban traffic with a fare share of of free flow lines, 30$<$v$<$45 km h$^{-1}$, 1.5 stops per km</td>
  </tr>
    <tr>
    <td style="text-align:center">c</td>
    <td style="text-align:center">Normal urban traffic</td>
    <td style="text-align:center">Urban traffic with some degree of congestion, v$<$15 km h$^{-1}$, 2 stops per km</td>
  </tr>
    <tr>
    <td style="text-align:center">d</td>
    <td style="text-align:center">Stagnating urban traffic</td>
    <td style="text-align:center">Urban traffic with a high degree of congestion, 15$<$v$<$30 km h$^{-1}$, 10 stops per km</td>
  </tr>

</table>
</body>
</html>

A street segment is assigned a single speed type as A, B, C or E, and a fraction of stagnating traffic ($F_s$):

\begin{equation}
E = \frac{1000}{24 \cdot 3600}\bigg[\left(f_l \cdot N_l + f_m \cdot N_m + f_h \cdot N_h \right)\left(1 - F_s\right) + \left(f_l \cdot N_l + f_m \cdot N_m + f_h \cdot N_h\right) F_s  \bigg].
\end{equation}

The associated emission factors are given in Table 2.

<html>
<head>
<style>
table, th, td {
  border: 1px solid black; 
  border-collapse: collapse;
}
th, td {
    padding: 15px; 
}
</style>
</head>
<body>
<p>Table 2: Emission factors $E_l$, $E_m$ and $E_h$ of NO$_x$, PM$_{10}$ and NO$_2$ as a function of a road type, speed type and vehicle category, as officially determined by the Dutch national government for 2013. The emission factors are updated annually.</p>
<table style="width:100%">
       <colgroup>
       <col span = '3'; style="width: 40%;">
       <col span = '3'; style="width: 20%;">
       <col span = '3'; style="width: 20%;">
       <col span = '3'; style="width: 20%;">    
    </colgroup>    
  <tr>
    <th style="text-align:center"; colspan="3">Code, description and speed </th>
    <th style="text-align:center"; colspan="3">Light vehicles</th> 
    <th style="text-align:center"; colspan="3">Medium heavy vehicles</th>
    <th style="text-align:center"; colspan="3">Heavy vehicles</th>
  </tr>
  <tr>
    <td style="text-align:center"><b>code</b></td>
    <td style="text-align:center"><b>description</b></td>
    <td style="text-align:center"><b>speed</b></td>  
    <td style="text-align:center"><b>NO$_x$</b></td>
    <td style="text-align:center"><b>PM$_{10}$</b></td>
    <td style="text-align:center"><b>NO$_2$</b></td>  
    <td style="text-align:center"><b>NO$_x$</b></td>
    <td style="text-align:center"><b>PM$_{10}$</b></td>
    <td style="text-align:center"><b>NO$_2$</b></td>  
    <td style="text-align:center"><b>NO$_x$</b></td>
    <td style="text-align:center"><b>PM$_{10}$</b></td>
    <td style="text-align:center"><b>NO$_2$</b></td>  
  </tr>
    <tr>
    <td style="text-align:center"><b>d</b></td>
    <td style="text-align:center"><b>stagnating urban traffic</b></td>
    <td style="text-align:center">$<$15</td>  
    <td style="text-align:center">0.6</td>
    <td style="text-align:center">0.049</td>
    <td style="text-align:center">0.13</td>  
    <td style="text-align:center">13</td>
    <td style="text-align:center">0.3</td>
    <td style="text-align:center">0.74</td>  
    <td style="text-align:center">19.1</td>
    <td style="text-align:center">0.34</td>
    <td style="text-align:center">0.9</td>  
  </tr>
  <tr>
    <td style="text-align:center"><b>c</b></td>
    <td style="text-align:center"><b>normal urban traffic</b></td>
    <td style="text-align:center">15-30</td>  
    <td style="text-align:center">0.36</td>
    <td style="text-align:center">0.043</td>
    <td style="text-align:center">0.09</td>  
    <td style="text-align:center">7.9</td>
    <td style="text-align:center">0.22</td>
    <td style="text-align:center">0.43</td>  
    <td style="text-align:center">11.6</td>
    <td style="text-align:center">0.24</td>
    <td style="text-align:center">0.53</td>  
  </tr>
  <tr>
    <td style="text-align:center"><b>e</b></td>
    <td style="text-align:center"><b>urban traffic with reduced congestion</b></td>
    <td style="text-align:center">30-45</td>  
    <td style="text-align:center">0.42</td>
    <td style="text-align:center">0.042</td>
    <td style="text-align:center">0.09</td>  
    <td style="text-align:center">5.4</td>
    <td style="text-align:center">0.18</td>
    <td style="text-align:center">0.28</td>  
    <td style="text-align:center">7.9</td>
    <td style="text-align:center">0.19</td>
    <td style="text-align:center">0.35</td>  
  </tr>
  <tr>
    <td style="text-align:center"><b>b</b></td>
    <td style="text-align:center"><b>provintial road</b></td>
    <td style="text-align:center">60</td>  
    <td style="text-align:center">0.24</td>
    <td style="text-align:center">0.023</td>
    <td style="text-align:center">0.07</td>  
    <td style="text-align:center">4.7</td>
    <td style="text-align:center">0.13</td>
    <td style="text-align:center">0.27</td>  
    <td style="text-align:center">6.4</td>
    <td style="text-align:center">0.13</td>
    <td style="text-align:center">0.3</td>  
  </tr>       
</table>
</body>
</html>

## 3. From emission to concentration: Ventilation

### 3.1 Budget equation

In section 2, the methodology for computing the emission rates in street canyons has been explained. But ultimately the NSL-monitoring  is intended to simulate concentrations. The step from emissions to concentration is by realising that the emissions are into a volume $V = l\times w \times h$. The mass balance of the pollutant in the volume is:
 
\begin{equation}
\frac{dC}{dt} = \frac{E\cdot l+F_{in}-F_{out}}{V}.
\end{equation}

This is to say: the concentration changes in time $\frac{dC}{dt}$ ($\mu$g m$^{-3}$ s$^{-1}$) are caused by emission ($E\cdot l$, $\mu$g s$^{-1}$) into the box, and by the difference between in- and outflow ($F_{in}-F_{out}$, $\mu$g s$^{-1}$). Since $C = \frac{m}{V}$, we can multiply both sides of the equation with $V$ to get:
 
\begin{equation}
\frac{dC}{dt} = \frac{E}{w\cdot h}+ \frac{F_{in}-F_{out}}{V} = \frac{E}{w\cdot h} + T.
\end{equation}

For simplicity we call $\frac{F_{in}-F_{out}}{V} = T$ (Transport). Considering that the efficiency of transport depends on the difference in concentration between the background and the box ($C_{bg}-C_{box}$, $\mu$g m$^{-3}$) and that the units of $T$ are $\mu$g m$^{-3}$ s$^{-1}$, we can write $T$ as:
 
\begin{equation}
T = \frac{C_{bg}-C_{box}}{\tau_v},
\end{equation}

where $\tau_v$ is the time constant of ventilation. If we assume steady state ($\frac{dC}{dt}=0$, the concentration in the box has reached an equilibrium):
 
\begin{equation}
\frac{dC}{dt} = 0 = \frac{E}{w\cdot h}+\frac{C_{bg}-C_{box}}{\tau_v}.
\end{equation}

Now, we can solve the equation for $C_{box}$:
 
\begin{equation}
C_{box} = C_{bg}+\frac{E\cdot \tau_v}{w\cdot h}.
\end{equation}

With this equation NSL-monitoring simulates the concentration of the pollutant in the box. It assumes that the concentration inside the box depends on the background concentration, on the local emissions, and on the ventilation-time constant. The background concentration is taken in NSL-monitoring from the GCN map of the Netherlands (Grootschalige Concentratiekaart Nederland, Largescale Concentration map Netherlands), which is prepared on an annual basis by the RIVM (RijksInstituut voor Volksgezondheid en Milieu, National Institute for Public health and the Environment) in Bilthoven. In the next section we will consider several options for defining $\tau_v$.

### 3.2 Ventilation rate

<br>
<div align=center>
<figure>
  <img src="Figure2_StreetCanyon.png", width="700px" height="600px">
  <figcaption> <i>Figure 2: Visualisation of the volume of the box and the ventilation rate.</i></figcaption>
</figure>
</div>
<br>

Figure 2 schematically shows the ventilation of the street canyon. In this example it is clear that the volume of the street is defined by the height of the buildings and the width of the street, and that the transport of pollutants occurs through the horizontal plane at the top of the volume. In this case, the ventilation takes place by means of turbulent diffusion. The ventilation time constant can be written as:
 
\begin{equation}
\tau_v=\frac{h^2}{2K_z},
\end{equation}

where $h$ is the height of the box and $K_z$ the turbulent diffusion coefficient. $K_z$ is a function of wind speed and the local surface roughness. The presence of vegetation is accounted for by the tree factor (next section). In a typical situation $h = 20$ m, $K_z = 3$ m$^2$ s$^{-1}$, and $\tau_v = 400/6 = 67$ s.

Figure 2 shows an example of a street canyon with tall buildings. This situation is also shown schematically in Figure 3a. In many rural and suburban situations the buildings on either side of the street are absent, intermittent and/or much lower (Figure 3b). In that case, the ventilation is not through the vertical plane at the top of the (ill-defined) box, but through the lateral boundaries. The ventilation time constant can be expressed as:
 
\begin{equation}
\tau_v=\frac{w}{u},
\end{equation}

assuming that the wind is perpendicular to the street. In a typical situation where  $w = 50$ m and $u = 2$ m s$^{-1}$, $\tau_v = 25$ s, which is almost one-third of the $\tau_v$ in the situation with exchange through the top boundary. The concentrations will therefore be much lower in this situation. Obviously, in practise it makes a difference if the wind is really perpendicular to the street or more parallel. NSL-monitoring uses geographically explicit, annual average wind speeds to calculate the concentrations.

<br>
<div align=center>
<figure>
  <img src="Figure3_Buildings.png", width="800px" height="700px">
  <figcaption> <i>Figure 3: a) Ventilation occurs through the top boundary of the box. b) Ventilation occurs through the lateral boundaries of the box.</i></figcaption>
</figure>
</div>
<br>


### 3.3 Tree factor

Figure 2 schematically shows that trees disturb the normal refreshment of the air. As a consequence the emitted pollutants remain below the tree crown longer, and the concentration increases. The NSL-monitoring allows to correct for this effect by introducing the tree factor $f_T$, which is 1.00 for an open street without trees, 1.25 for streets with some trees and 1.5 for streets with a closed tree crown (Figure 4). The tree factor is applied to increase the concentration in the box:
 
\begin{equation}
C_{box}=C_{bg}+\frac{E\cdot \tau_v}{w \cdot h}f_T.
\end{equation}

In this approach it is assumed that trees suppress the ventilation rate more than they filter the pollutants from the air onto the leaves, as such it simulates a net effect. 

<br>
<div align=center>
<figure>
  <img src="Figure4_TreeFactor.png", width="600px" height="500px">
  <figcaption> <i>Figure 4: Guideline for determining the tree factor. </i></figcaption>
</figure>
</div>
<br>

### 3.4 Road type

The refreshment rate depends on the surface area of the street canyon, $A = w \cdot h$. To avoid having to measure the height of the neighbouring buildings on each road segment, NSL-monitoring uses a fixed classification of road types, each with a specific ratio of $l/h$ (Figure 5). As the width and the average wind speed of the street segments is available in the NSL database, the surface area and the refreshment rates can be computed.

For roads outside urbanisations, specific codes have been introduced: 92 for provincial, inter-communal  roads and 93 for highways, and 94 for highways with strict enforcement of the maximum speed by means of road section control or automated cameras. These codes are explicitly used in combination with maximum speed for light and heavy vehicles.

<br>
<div align=center>
<figure>
  <img src="Figure5_RoadType.png", width="400px" height="300px">
  <figcaption> <i>Figure 5: Road types in NSL-monitoring. </i></figcaption>
</figure>
</div>
<br>

### 3.5.	Other complicating factors

#### 3.5.1.	Crossings
Older versions of the NSL-monitoring accounted for the presence of crossings near the road segment. Near crossings vehicles (should) reduce speed, which would reduce the emission factors by a few percent. This functionality is no longer maintained in the current version of the NSL-monitoring.

#### 3.5.2.	Parking movements
Parking cars often run idle for some time, and often have cold engines. This enhances the emission of some volatile organic compounds, such as benzene. At larger parking lots this may cause exceedence of air quality standards. This functionality is not considered in this practical. 

#### 3.5.3.	Tunnel factor
Road segments may be in a tunnel, where air is refreshed by means of forced ventilation. The ventilation has outlets on either side of the tunnel, so that the pollutant concentration is much higher near the outlets. Road segments near tunnels may be assigned a tunnel factor, which is 0 for roads in tunnels (emission does not affect the air quality of the ambient air outside the tunnel, 1 for normal roads and > 1 for road segments near tunnel outlets.

### 3.6.	Probability of exceedance of air quality standards

NSL-monitoring simulates the annual average concentration of pollutants. This is however, only one of the variables of interest. EU air quality standards often specify a maximum for the annual average concentration, as well as a maximum number of days when a specified daily average concentration may be exceeded. NSL-monitoring computes the exceedances as a function of the average concentration, the standard, and average meteorological conditions. The theory behind the computational routine is that concentration exceedance probabilities are distributed logarithmically around the average concentration: the higher the exceedance, the lower the probability. Although we do not go into detail here, it is worth remembering that the probability of exceedance is estimated in a statistical sense.

# 4. NSL model in practice

The NSL model calculates the annual average concentrations at a receptor point as the background concentration + a local contribution (section 3.1):
\begin{equation}
C_{box} = C_{bg}+\frac{E\cdot \tau_v}{w\cdot h}.
\end{equation}


Thus, to calculate the annual averages of concentrations of air pollutants, the NSL model  needs:
- the background concentration C$_{bg}$ at the receptor point
- the emissions along the street segment
- the ventilation rate

## 4.1 Background concentration

The large-scale background concentration at the location of the receptor point is derived from the Large Scale Concentration Maps Netherlands (GCN: Grootschalige Concentratiekaarten Nederland, https://geodata.rivm.nl/gcn/), which are generated by the RIVM. In the files provided by the RIVM two values are available: one refers is the GCN concentration at the location of the receptor point, while a second refers to the GCN concentration when all contribution from motorways, Hoofdwegennet (HWN in Dutch, that are located up to a distance of 3.5 km are substracted from the GCN-concentration. GCN maps are provided for horizontal grid boxes of 1 km to 1 km and are available for 2020, 2025 and 2030. 

In order to obtain the background concentration at the receptor point, you need to tell the NSL model the location of the receptor point and the year for which you want to do the calculations:
1. RDW-X: x-coordinates according to the National Coordinate System (See <font color="blue">Box</font> below)
2. RDW-Y: y-coordinates according tot National Coordinate System. 
3. Year for which the calculations should be done.

Values can be provided using dropdown menu (year) and via input in text boxes (RDW-X and RDW-Y).

### <font color="blue">Box: The National Coordinate System (Rijksdriekhoekscoordinaten)
<font color="blue">The National Coordinate System defines X towards the East and Y towards the North in m of a hypothetical point 120 km SW of Paris. As a result, X and Y are always positive and Y is always larger than X. More about the system can be read on https://en.wikipedia.org/wiki/Rijksdriehoeksco%C3%B6rdinaten.</font>

### <font color="red">How to obtain the coordinates for your street location?
<font color="red">
    
To obtain the coordinates of your street location:
1. Go to https://nsl-monitoring.nl/viewer/
1. Zoom into the location of your street
1. In the menu on the right select:
    - 'Rekenpunten tonen' (show receptor points)
    - 'Wegvakken tonen' (show road segments)
    - 'Overdrachtslijnen tonen' (show transfer lines)
1. You should now see the receptor points as circles and the road segments as orange lines.   
1. In the menu on the top left: click the 'select' tool
1. Draw a rectangle around the recepor points, road segments and transfer lines of your interest.
1. At the top of the browser window, you will now see three active boxes:
    - 'Rekenpunten' (receptor points)
    - 'Segmenten' (road segments)
    - 'Overdrachtslijnen' (transfer lines)
1. Click 'Rekenpunten' and select the receptor point you want in the pop up screen at the bottom.
1. Click 'Kenmerken' (Characteristics) and scroll down until you see the X-coordinate and Y-coordinate in m.
1. Write down the X-and Y-coordinate (no need to be more precies than 1000 m, since the background concentration map has a 1 km resolution) in the cell below **AND FINISH WITH _ENTER_**.

</font>

In [8]:
import CARII
import ipywidgets as widgets
import pandas as pd
from calculations import calc_gcn_meteo_emissfacts, calc_Emiss, calc_concs
%matplotlib inline

In [9]:
recep = CARII.receptor()

BoundedFloatText(value=133382.0, description='RDW_x (m East):', layout=Layout(width='300px'), max=280000.0, st…

BoundedFloatText(value=454361.0, description='RDW_y (m North):', layout=Layout(width='300px'), max=640000.0, m…

Dropdown(description='Year', layout=Layout(width='300px'), options=(2020, 2025, 2030), style=DescriptionStyle(…

With the information given above, you can derive:
- the GCN background concentrations (both with HWN and without HWN),  
- 10 year average wind speed in that region
- the regional factor which is calculated from the ten-year average wind speed.

In [10]:
#### fun the function as given above)
calc_gcn_meteo_emissfacts(recep, CARII)

# print out the different attributes of the receptor point.
for species in ['NOx','NO2','PM10','PM25']:
   print('[%-4s]_gcn %-9s HWN %5.1f %6s'%(species, 'including',recep.gcn[species+'_HWN'  ],' \u03BCg/m3'))
   print('[%-4s]_gcn %-9s is: %5.1f %6s'%(species, 'without',  recep.gcn[species+'_noHWN'],' \u03BCg/m3'))
print('%-45s %5.1f  m/s'%('The wind speed is: ',recep.ws))
print('%-45s %5.1f  s/m'%('Fregion is:',recep.Fregio))


FileNotFoundError: [Errno 2] No such file or directory: '/home/lcur0000/JHL_data/MAQ11306_P03_NO2_NSL/conc_NOx_RR2020_2002.asc'

## 4.2 Emissions along the street segment

Now that we have the background concentrations, we need to quantify the local contribution from traffic at the receptor point. For this reason we first calculate the emissions. The NSL model needs the following parameters:

The number of separate road tracks<br>
If https://nsl-monitoring.nl/viewer shows the road of your interest as a single road segment, select '1' from the dropdown menu (this is the most common option). If there are multiple road segments, select '2' and give the parameters for each road segment.


For each road segment the following information needs to be provided in the cell below:
1. SCM road type: SCM=Standard Calculation Method (NL: SRM). This parameter can be either 'motorway' or 'non-motorway'. For SCM1, only non-motorways can be considered. 'Non-motorway' is the most common choice for urban roads.<br>
<font color="red">Select 'Rekenpunten' (Receptor points) at the top of the screen, select the correct one in the popup screen at the bottom and check whether it shows a 'SRM 1' (='non-motorway') tab or a 'SRM 2' (= 'motorway') tab. </font> 

2. Speed Type (NL: Snelheidstype): the fluidity type of the of the traffic. There are five categories: 
    - (a: motorway)
    - b: provincial road
    - e: urban traffic with less stagnation (NL: 'stadsverkeer met minder congestie').
    - c: normal urban traffic (NL: 'normaal stadsverkeer')
    - (d: urban traffic with congestion (NL: 'stagnerend stadsverkeer') )

    In this notebook application, only types b, e and c are implemented. For a motorway, please use type b and for urban traffic with congestion, use type d. This will affect the emission factors somewhat, but it will still be workable.<br>
<font color="red">In the 'SRM 1' or 'SRM 2' tab in the popup screen at the bottom (see previous step), check the letter behind 'Snelheidstype' (Speed type). </font>   
3. f_s: the fraction of daily traffic that should be considered as congested traffic. 
<br>
<font color="red">In the 'Intensiteiten' (Intensities) tab in the popup screen at the bottom (see step 1), check the 'Congestie' (Stagnation factor) number. In NSL-monitoring it is specified for light, middle and heavy traffic separately. Here you need to enter a single number for all.</font>
4. N_l: the number of light weight vehicles per day<br>
<font color="red">In the 'Intensiteiten' (Intensities) tab in the popup screen at the bottom (see step 1), check the 'Intensiteit' (Intensity) number for 'licht verkeer' (light traffic)</font>.
5. N_m: the number of medium-weight vehicles per day<br>
<font color="red">In the 'Intensiteiten' (Intensities) tab in the popup screen at the bottom (see step 1), check the 'Intensiteit' (Intensity) number for 'middelzwaar verkeer' (middle heave traffic)</font>.
6. N_h: the number if heavy-weight vehicles per day<br>
<font color="red">In the 'Intensiteiten' (Intensities) tab in the popup screen at the bottom (see step 1), check the 'Intensiteit' (Intensity) number for 'zwaar verkeer' (heavy traffic)</font>.
7. N_b: the number of buses per day<br>
<font color="red">In the 'Intensiteiten' (Intensities) tab in the popup screen at the bottom (see step 1), check the 'Intensiteit' (Intensity) number for 'bus' (Busses)</font>. Note that busses are normally included in heavy traffic. Only if a muncipality has taken action to contribute to cleaner air by having cleaner busses, they will be included in this number.
8. Road Type (Wegtype): the type of the road.<br>
<font color="red">Four types are defined (see figure below for a definition). In the notebook we do not distinguish 3a from 3b.</font>

<img src="FigureRoadType.png" height="200px" width="300px">

9. f_t: Tree factor (NL: bomenfactor)There are three classes for the tree factor:
    - 1: no trees
    - 1.25: seperate trees
    - 1.5: trees that have overlapping crowns<br>
<font color="red">
    You can either:
    - At the top of the screen, select 'Overdrachtslijnen', in the popup screen at the bottom, select the correct one and look for 'bomenfactor (tree factor). Note: for some receptorpoints, the tree factor is not given.
    - Estimate the tree factor from the figure below
<img src="FigureTreeFactor.png" width="400px"></font>
10. Distance (m): distance between the center line of the road part and the receptor point.<br>
<font color="red">The distance from the road is not provided in NSL-monitoring. You can estimate it yourself. A typical value would be 10-15 m.</font>

In [3]:
recep.define_rijbanen()

Dropdown(description='Nr road tracks', layout=Layout(width='300px'), options=(1, 2), style=DescriptionStyle(de…

Give parameters for road section:  1


Dropdown(description='SCM Road Type', layout=Layout(width='400px'), options=('non-motorway', 'motorway'), styl…

Dropdown(description='Speed Type', layout=Layout(width='400px'), options=('urban normal', 'urban less congesti…

BoundedFloatText(value=0.0, description='f_s Fraction Stagnation (-)', layout=Layout(width='400px'), max=1.0, …

...

BoundedFloatText(value=65.0, description='N_h Intensity Heavy (veh/24h):', layout=Layout(width='400px'), max=1…

BoundedFloatText(value=0.0, description='N_b Intensity Buses (veh/24h)', layout=Layout(width='400px'), max=100…

Dropdown(description='Road Type', layout=Layout(width='400px'), options=('1', '2', '3', '4'), style=Descriptio…

Dropdown(description='f_t Tree factor', layout=Layout(width='400px'), options=(1.0, 1.25, 1.5), style=Descript…

FloatText(value=13.0, description='Distance (m)', layout=Layout(width='400px'), style=DescriptionStyle(descrip…

---
With these input parameters, the emissions on the road segment of your interest can be calculated:



In [4]:
calc_Emiss(recep)
for id,rijbaan in enumerate(recep.rijbanen):
    for species in ['NOx','NO2','PM10','PM25']:
        print('Road segment %2d: E_%-4s = %5.1f \u03BCg/m/s'%(id+1,species,rijbaan.emiss[species]))
    

Road segment  1: E_NOx  =  40.9 μg/m/s
Road segment  1: E_NO2  =   6.8 μg/m/s
Road segment  1: E_PM10 =   2.9 μg/m/s
Road segment  1: E_PM25 =   1.0 μg/m/s


## 4.3 Contribution from nearby motorways

The GCN background concentrations are given with and without the contribution of motorways closer than 3.5 km. If large scale concentrations are used without the motorways, the contribution of all roads that are calculated according to Standard Calculation Method 2 (SCM 2) need to be added.
However, contributions by SCM 2 roads can also be prescribed by the user. Contributions need to be provided for NO$_x$, PM$_{10}$ and PM$_{2.5}$, while also a value of the fraction of direct emissions of NO2 over all NOx emissions for SCM 2 roads need be to given.

<font color="red">
    
- At the top of the screen, select 'Rekenpunten' (receptor points), in the popup screen at the bottom, select the receptor point ('rekenpunt') of your interest. Go to 'Concentraties' (Concentrations).
- FNO2_srm2: can be found in the line 'SRM2-bijdrage fractie directe uitstoot NO2 [-]' (SCM 2 contribution fraction direct emissions [-])
- [NOx]\_SRM2, [PM10]\_SRM2, [PM2.5]\_SRM2: 'SRM2-bijdrage [ug/m3]' (SCM contribution [ug/m3]) in the corresponding column.
- **Hit _ENTER_ after each number you have entered. Check the values later on in the cell where the final concentrations are given.**    
</font>

In [7]:
recep.get_srm2()

BoundedFloatText(value=0.199, description='FNO2_srm2:', layout=Layout(width='300px'), max=1.0, step=0.01, styl…

BoundedFloatText(value=1.776, description='[NOx]_SRM2:', layout=Layout(width='300px'), style=DescriptionStyle(…

BoundedFloatText(value=0.095875, description='[PM10]_SRM2:', layout=Layout(width='300px'), style=DescriptionSt…

BoundedFloatText(value=0.04, description='[PM2.5]_SRM2:', layout=Layout(width='300px'), style=DescriptionStyle…

Now that we have the emissions of the different road parts for the different species, the contribution to NO2, PM10, PM2.5 of Srm2 road, and the fractio of direct emissions of NO2 over all emission of NOx for Srm2 roads, we can calculate the contribution of traffic on the concentration of the different species and the total concentrations. 

Run the function as given above for the receptor point and print out the results

In [8]:
calc_concs(recep)

print('%17s | %5s | %12s        | %5s |'%('','bg','local',''))
print('%17s | %5s | %5s  %5s  %5s | %5s |'%('species (units)','gcn','SCM1','SCM2','sum','total'))
print('-------------------------------------------------------')
for species in ('NO2','NOx','PM10','PM2.5'):
    conc_totl = recep.Cb  [species]
    conc_finl = recep.Ctot[species]
    if species == 'PM2.5':
        conc_gcn   = recep.gcn['PM25_noHWN']
    else:
        conc_gcn   = recep.gcn[species+'_noHWN']
    if species == 'NO2':
        conc_srm1 = -99.
        conc_srm2 = -99.
    else:
        conc_srm1 = recep.Cb  [species + '_srm1']
        conc_srm2 = recep.Cb  [species + '_srm2']
        
    print('[%-7s] (\u03BCg/m3) | %5.1f | %5.1f  %5.1f  %5.1f | %5.1f | '%(species,conc_gcn,conc_srm1,conc_srm2,conc_totl,conc_finl))
    

                  |    bg |        local        |       |
  species (units) |   gcn |  SCM1   SCM2    sum | total |
-------------------------------------------------------
[NO2    ] (μg/m3) |  19.0 | -99.0  -99.0    1.4 |  20.5 | 
[NOx    ] (μg/m3) |  24.9 |   4.0    0.0    4.0 |  29.0 | 
[PM10   ] (μg/m3) |  18.0 |   0.3    0.0    0.3 |  18.3 | 
[PM2.5  ] (μg/m3) |  11.3 |   0.1    0.0    0.1 |  11.4 | 


### 99. Exercises

### Model
- how large is the background concentration?
- how large is the local contribution?

### Observations
- can you estimate the background concentration from your observations?
- how large is the street concentration?
- can you estimate the local contribution from the difference?
- what was the wind speed during your observations (look at KNMI day report or so)
- 

### Comparison
- can you adjust the model calculations to the situation 
